# Przetwarzanie Grafiki i Muzyki - laboratorium nr 7

## Binaryzacja.

Binaryzacja polega na zamianie obrazów monochromatycznych na obrazy binarne. Przekształcenie to jest prawie zawsze wykorzystywane w analizie obrazów, gdyż wiele operacji może być przeprowadzonych wyłącznie na obiektach binarnych.

## Zadanie 1 (1 punkt)

Proszę zaimplementować binaryzację z zadanym poziomem ( a ).

$$
J(i,j) = \left\{ \begin{array}{ll}
0 & \textrm{gdy } J(i,j)<a \\
255 & \textrm{gdy } J(i,j) \geq a 
\end{array} \right.
$$

  * Proszę zastosować do zdjęcia "LENA_512.jpg". 
  * Proszę wykonać na Luminancji (składowa Y z formatu YCbCr). 

## Zadanie 2 (1 punkt) 

Proszę zaimplementować binaryzację z zadanym przedziałem ($[a,b]$):

$$
J(i,j) = \left\{ \begin{array}{ll}
255 & \textrm{gdy } J(i,j) \in \left[a,b\right] \\
0 & \textrm{w przeciwnym przypadku} 
\end{array} \right.
$$

Zastosuj tę binaryzację do obrazu "LENA_512.jpg".

# Zadanie 3 (2 punkty)

Prosze użyć algorytm binaryzacji Otsu do zdjęcia "gazeta.jpg". 

![alt](https://raw.githubusercontent.com/przem85/PGiM/master/img/gazeta.jpg)

Proszę wykonać na Luminancji (składowa Y z formatu YCbCr).

# Zadanie 4 (3 punkty)

Proszę zaimplementować algorytm binaryzacji Bernsena. Metoda Bernsena to algorytm wyznaczania progu lokalnego w oparciu o otoczenie piksela (zwykle kwadratowe okno o nieparzystej szerokości). 
Próg ustawiany jest na wartość średnią najjaśniejszego i najciemniejszego piksela.

  * Proszę użyć algorytm binaryzacji Bernsena do zdjęcia "gazeta.jpg".
  * Proszę wykonać na Luminancji (składowa Y z formatu YCbCr).

Algorytm Brensena:
1. Przekształcamy oryginalny obrazek do skali szarości. Możemy teraz myśleć o obrazku, jak o pewnej funkcji $I\left(x,y\right): \mathbf{N}^2 \rightarrow \mathbf{N}$ (zakładając, że poruszamy się po świecie liczb naturalnych).
2. Wokół każdego piksela bierzemy kwadrat o nieparzystej długości $a>0$ i wyliczamy wartość minimalną oraz maksymalną z intensywności pikseli w każdym takim kwadracie. Formalnie, dla każdego punktu $\left(x,y\right)\in\mathbf{N}^2$ i ustalonego $a>0$ wyliczamy:

\begin{align}
  v_{min}\left(x,y;a\right) &= \min{ \left( I\left(i,j\right): x-a \leq i \leq x+a, \quad y-a \leq j \leq y+a \right) } \\
  v_{max}\left(x,y;a\right) &= \max{ \left( I\left(i,j\right): x-a \leq i \leq x+a, \quad y-a \leq j \leq y+a \right) }
\end{align}
3. Dla każdego kwadratu wyliczamy próg $T$ jako $T = \frac{v_{min}\left(x,y;a\right) + v_{max}\left(x,y;a\right)}{2}$.

# Zadanie 5 (3 punkty)
Progowanie mieszane przebiega podobnie, jak progowanie lokalne (metoda Bernsena), z tym, że jeśli średnia lub mediana lokalna dla danego piksela odbiega o więcej niż ustalony próg (program pobiera tą wartość jako parametr) od wartości globalnej (wyznaczona za pomocą Otsu), piksel progowany jest wartością globalną.

  * Prosze użyć algorytm progowania mieszanego do zdjęcia "gazeta.jpg".
  * Proszę wykonać na Luminancji (składowa Y z formatu YCbCr).

  Algorytm progowania mieszanego:
  1. Wyznaczamy wartość progu globalnego przy pomocy binaryzacji Otsu. Oznaczmy ten próg jako $P$.
  2. Podobnie, jak w algorytmie Brensena, przekształcamy oryginalny obrazek do skali szarości.
  3.  Wokół każdego piksela bierzemy kwadrat o nieparzystej długości $a>0$ i wyliczamy wartość minimalną oraz maksymalną z intensywności pikseli w każdym takim kwadracie. Formalnie, dla każdego punktu $\left(x,y\right)\in\mathbf{N}^2$ i ustalonego $a>0$ wyliczamy:

\begin{align}
  v_{min}\left(x,y;a\right) &= \min{ \left( I\left(i,j\right): x-a \leq i \leq x+a, \quad y-a \leq j \leq y+a \right) } \\
  v_{max}\left(x,y;a\right) &= \max{ \left( I\left(i,j\right): x-a \leq i \leq x+a, \quad y-a \leq j \leq y+a \right) }
\end{align}

4. Dla każdego kwadratu wyliczamy próg $T$ jako $T = \frac{v_{min}\left(x,y;a\right) + v_{max}\left(x,y;a\right)}{2}$. Jeżeli jednak wartość tak wyliczonego progu $T$ różni się o więcej niż $\alpha > 0$ (parametr ten powinien być wczytywany jako argument funkcji) od progu globalnego $P$, to wtedy piksel progowany jest wartością globalną. Formalnie, jeśli $\left| T - \alpha \right| > P$, to wtedy jako próg lokalny przyjmujemy próg globalny $P$.